In [182]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [183]:
import pandas as pd
import numpy as np
import seaborn as sns

from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

from scipy import stats
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


!pip install openpyxl

In [184]:
dfx = pd.read_csv('../input/ratio-real-estate/Data Financial Ratio Real Estate.csv')
dfx.info()

In [185]:
dfx.describe()

In [186]:
dfx.head()

In [187]:
dfx.drop(["Emiten","Industri","Company","Category","No"], axis=1, inplace=True)
dfx.info()

In [188]:
dfx

PCA

In [189]:
scaler = StandardScaler()

dfx = scaler.fit_transform(dfx)

In [190]:
dfx

In [191]:
pca = PCA(n_components=2)

principal_components = pca.fit_transform(dfx)

df1 = pd.DataFrame(data = principal_components, columns=['PC1','PC2'])

In [192]:
per_var = np.round(pca.explained_variance_ratio_ * 100, decimals =1)

label = ['PC' + str(x) for x in range (1, len(per_var)+1)]

plt.bar (x=range(1, len(per_var)+1),height=per_var,tick_label=label)
plt.ylabel('percentage of explained variance')
plt.xlabel('principal components')
plt.title('scree plot')
plt.show()

Clustering

In [193]:
df1['Kategori'] = pd.NaT

In [194]:
df1.describe()

In [195]:
df1.head()

In [196]:
plt.figure(figsize=(15, 15))
sns.pairplot(df1, vars = df1.columns[1:13]);

In [209]:
dfy = pd.read_csv('../input/ratio-real-estate/Data Financial Ratio Real Estate.csv')
df_cl = dfy[['DER','ROA','Category']]
df_cl.describe()

In [210]:
plt.figure(figsize=(18, 4))
plt.xticks(rotation=45)
sns.scatterplot(x='Emiten', y='DER', 
                data=dfy, s=76)

In [211]:
plt.figure(figsize=(18, 4))
plt.xticks(rotation=45)
sns.scatterplot(x='Emiten', y='ROA', 
                data=dfy, s=76)

In [207]:
print(f'Jumlah baris sebelum memfilter outlier: {len(df1)}')

filtered_entries = np.array([False] * len(df1))
for col in ['PC1']:
    zscore = abs(stats.zscore(df1[col]))
    filtered_entries = (zscore < 3) | filtered_entries
    
df1 = df1[filtered_entries]

print(f'Jumlah baris setelah memfilter outlier: {len(df1)}')

In [208]:
df1.describe()

PC1 DER
PC2 ROA

In [212]:
feats = ['PC1', 'PC2']
X = df1[feats].values
y = df1['Kategori'].values

from sklearn.preprocessing import StandardScaler
standardscaler = StandardScaler().fit_transform(X)
df2 = pd.DataFrame(data = standardscaler, columns = feats)
df2.describe()

In [213]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

inertia = []

for i in range(1, 11):
  kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
  kmeans.fit(df2.values)
  inertia.append(kmeans.inertia_)

plt.figure(figsize=(12, 6))
plt.title('Real Estate Data and Ratio 2020')
plt.xlabel('No. of Cluster')
plt.ylabel('Intertia')
plt.plot(inertia) #Elbow Method
plt.show()

In [214]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4, init='k-means++', max_iter=300, n_init=10, random_state=0)
kmeans.fit(df2.values)

In [217]:
df2['target'] = y
df2['fit'] = kmeans.labels_

In [219]:
plt.rcParams["figure.figsize"] = (8, 8)
sns.scatterplot(x='PC1', y='PC2', hue=df2.target.to_list(), data=df2);

In [221]:
df2.head(5)

In [223]:
df2.insert(0,"Kategori",np.nan,True)
df2.loc[df2['fit'] == 0, 'Kategori'] = 0
df2.loc[df2['fit'] == 1, 'Kategori'] = 1
df2.loc[df2['fit'] == 2, 'Kategori'] = 2
df2.loc[df2['fit'] == 3, 'Kategori'] = 3

df2.head(5)